# Exploratory Data Anaysis

Before we begin with the *EDA* we need to read the *metadata*. If there is not (as is our current case) we will create after the *EDA*.

We need highlight that the analysis the data we will be have an economics approach, but it can apply any approach.

In order to track the transformations that we apply to our data, each time we apply these transformation we will save all files and will do commit (*snapshoot*) using git. This useful by reasons of *reproducibility* 


In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

Data Source:

* [Principles of Econometrics](http://www.principlesofeconometrics.com)

In [5]:
mroz = pd.read_csv('http://www.principlesofeconometrics.com/poe5/data/csv/mroz.csv')

This data can be taken as a *sample* or as the *population*. But almost always in a company, the data is the *population of customers*. So, the data will be considered as *population*. This *assumption* must point out in the metadata.

In [8]:
mroz

,taxableinc,federaltax,hsiblings,hfathereduc,hmothereduc,siblings,lfp,hours,kidsl6,kids618,...,hage,heduc,hwage,faminc,mtr,mothereduc,fathereduc,unemployment,largecity,exper
0,12200,1494,1,14,16,4,1,1610,1,0,...,34,12,4.0288,16310,0.7215,12,7,5.0,0,14
1,18000,2615,8,7,3,0,1,1656,0,2,...,30,9,8.4416,21800,0.6615,7,7,11.0,1,5
2,24000,3957,4,7,10,2,1,1980,1,3,...,40,12,3.5807,21040,0.6915,12,7,5.0,0,15
3,16400,2279,6,7,12,5,1,456,0,3,...,53,10,3.5417,7300,0.7815,7,7,5.0,0,6
4,10000,1063,3,7,7,7,1,1568,1,2,...,32,12,10.0000,27300,0.6215,12,14,9.5,1,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
748,32000,4701,8,12,7,5,0,0,0,2,...,43,16,9.2715,28200,0.6215,10,10,9.5,1,5
749,18500,2720,4,12,12,8,0,0,2,3,...,33,12,4.8638,10000,0.7715,12,12,7.5,0,14
750,13000,1642,6,7,12,2,0,0,0,0,...,43,12,1.0898,9952,0.7515,10,3,7.5,0,4
751,17200,2447,2,7,10,2,0,0,0,0,...,55,8,12.4400,24984,0.6215,12,12,14.0,1,15
